In [1]:
import requests
from dotenv import load_dotenv
import os

url = "https://192.168.1.100:8000/"
load_dotenv()
ssl_cert_file = os.getenv("SSL_CERT_FILE_PATH")

In [2]:
response = requests.get(url, verify=ssl_cert_file)
if response.status_code == 200:
    print("Response from server:", response.json())
else:
    print("Error:", response.status_code, response.text)

Response from server: {'message': 'Hello, World!'}


In [3]:
username = "test_user"
tsv_file_path = "../datasets/training/user4.tsv"

with open(tsv_file_path, "r", encoding="utf-8") as file:
    tsv_data = file.read()
    
    
response = requests.post(
    url + 'train',
    params={"username": username},
    data=tsv_data,
    verify=ssl_cert_file,
)

if response.status_code == 200:
    print("Response from server:", response.json())
else:
    print("Error:", response.status_code, response.text)

Response from server: {'message': 'TSV data received successfully'}


In [4]:
username = "test_user"
tsv_file_path = "../datasets/inference/user4.tsv"

with open(tsv_file_path, "r", encoding="utf-8") as file:
    tsv_data = file.read()


response = requests.post(
    url + 'inference',
    params={"username": username},
    data=tsv_data,
    verify=ssl_cert_file,
)

if response.status_code == 200:
    print("Response from server:", response.json())
else:
    print("Error:", response.status_code, response.text)

Response from server: {'message': 'TSV data received successfully', 'score': 0.0845070406794548, 'prediction': 0}
